In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=3, suppress=True)

## General remarks

It appeals to me that it is better to define set of vectors as arrays. Vectors should be columnar
and it is conveniently to define them as 
- `v = np.array([[1,2,-3]]).T`

or equivalently (e.g `v = np.arange(5).reshape(-1,1))`

It is also convenient to represent basis $[e_1, e_2,\dots,e_n]$ as two-dimensional array: `np.hstack((e_1, e2, ..., e_n))`

where `e_j` is the j-th (columnar!) vector.

In [2]:
class Vector(np.ndarray):
    def __new__(cls, input_array):
        obj = np.asarray(input_array).reshape(-1,1)
        return obj

In [3]:
def proj(v, u):
    """
    Projects orthogonally v on u
    
    u, v - one dimensional vectors (n x 1 arrays)
    """
    # what if u == 0?
    return (u.T@v) / (u.T@u) * u

In [4]:
def GS(V, inplace=False):
    """
    Gramm-Schmidt ortogonalization
    
    V - n x m matrix - hstack of m n-dimensional vectors (n x 1 arrays).
    returns: n x m matrix of orthogonalized vectors
    
    if inplace==True V is modified; otherwise a new matrix is returned
    """
    U = np.array(V, copy=True)  # equivalent to np.copy(V)
    m = V.shape[1]
    for j in range(m):
        for i in range(j):
            U[:,j] -= proj(V[:,j], U[:,i])
    return U

In [3]:
def decompose_in_basis(vector, basis):
    """
    vector - n x 1 array
    basis - n x n array thought of as hstack of n x 1 dimensional creating basis of R^n
    returns - coeffs of vector in basis 
    """
    return np.linalg.solve(basis, vector)

In [4]:
def compose_in_basis(coeffs, basis):
    """
    computes vector having its coeffs in basis
    """
    return Vector(basis@coeffs)

## Remarks on linear transformations
In order to find (or build) the matrix of a linear transformation $F$ in a given basis ${\bf B} = \{e_1, e_2,\dots, e_n\}$ you must:
* decompose images $F(e_i)$ of basis vectors in the basis ${\bf B}$
* put sets of coefficients of that decomposition as consecutive columns of the matrix $M$.
Then, if you have the matrix $M$ and a vector $v$ you can compute the value $F(v)$ as follows:
* decompose $v$ in ${\bf B}$ to get coefficients $c_1, c_2,\dots, c_n$
* compute `M @ c` where `c` is a vector $[c_1, c_2,\dots,c_n]$ - this gives you coeffs of $F(v)$ in ${\bf B}$
* compute $F(v)$ using calculated coeffs and ${\bf B}$

In [94]:
# F = np.array([[1,2,3], [-2,0,0], [0,1,2]])

def F(vector):
    x, y, z = vector
    return Vector([2*x-y, x+z, -x+y+3*z])
    
e1, e2, e3 = Vector([1,3,-2]), Vector([0,1,2]), Vector([0,-3,1])

E = np.hstack((e1, e2, e3))    # basis as 2d-array

# matrix of F in basis E
MF = np.hstack((decompose_in_basis(F(e1), E), decompose_in_basis(F(e2), E),
                decompose_in_basis(F(e3), E)))
               
x = Vector([2,1,-3])
xE = decompose_in_basis(x, E)  # xE - set of coeffs of x in E

t = MF@xE  # set of coeffs of resulting F(xE) in basis E

# print(F(x))
# compose_in_basis(t, E)

### Let's assume we have two bases
Let ${\bf E, E_1}$ be two basis: ${\bf E} = [e_1, e_2,\dots, e_n]$, ${\bf E_1} = [e_1^{'}, e_2^{'},\dots, e_n^{'}]$.

Let $M$ be a two-dimensional array with **rows** built as follows: the first row is composed of coeffs of decomposition of $e_1^{'}$ in ${\bf E}$, the second row is composed of coeffs of decomposition of $e_2$ in ${\bf E}$ and so on.

Similarly, let $M_1$ be a two-dimensional array arising from decomposition of vectors of ${\bf E}$ in ${\bf E_1}$. One can easily verify that $M$ and $M_1$ are *reciprocally inverse*.

Now, if $[x_1,x_2,\dots,x_n]$ are coeffs of $x$ in ${\bf E}$ and $[x_1^{'},x_2^{'},\dots,x_n^{'}]$ are coeffs of $x$ in ${\bf E_1}$ then one can also check that

$[x_1^{'},x_2^{'},\dots,x_n^{'}] = [x_1,x_2,\dots,x_n]\circ M^{-1} = [x_1,x_2,\dots,x_n]\circ M_1$ (cf. above).

In [95]:
E = np.array([[1,2,3], [1,0,1], [0,-1,2]]).T
E1 = np.array([[1,1,1], [-1,-1,2], [1,2,0]]).T

M = np.vstack([decompose_in_basis(E1[:,j], E).reshape(1,-1) for j in range(3)])  # decompose vectors from E1 in E
M1 = np.vstack([decompose_in_basis(E[:,j], E1).reshape(1,-1) for j in range(3)])

# print(np.hstack([sum(M[k,j]*E[:,j] for j in range(3)).reshape(-1,1) for k in range(3)])) # recompose E1 from M

In [99]:
x = Vector([7,-1,3])
xE = decompose_in_basis(x, E)
xE1 = decompose_in_basis(x, E1)

print(xE)
print(xE1)

[[-1.]
 [ 8.]
 [-1.]]
[[11.]
 [-4.]
 [-8.]]


In [5]:
print(xE1.T@M)

NameError: name 'xE1' is not defined

In [22]:
B = np.array([[1,2,3], [0,-2,1], [-1,1,2]]).T
v4 = Vector([-2,3,1])

a = Vector([-2,0,3])
b = decompose_in_basis(compose_in_basis([-2,0,3], B), B)
np.linalg.norm(a-b)


6.294154347968077e-16